# 1

In [11]:
from helper import aoc
import numpy as np
year=2021
day=1
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )
prev=None
increased=0

values =list(map(lambda x:int(x.strip()), f.readlines()))
for l in values:
    #l=int(l.strip())
    if prev!=None and l>prev:
        increased += 1

    prev=l
print(increased)
lastThree=[]
increased=0
for l in values:
    if len(lastThree)>=3:
        prev=sum(lastThree)
        lastThree.append(l)
        lastThree.pop(0)
        if sum(lastThree)>prev:
            increased+=1
    else:
        lastThree.append(l)
print(increased)


1121
1065


# 2

In [16]:
from helper import aoc
year=2021
day=2
fileName=aoc.init(day,year)
f = open( fileName  , 'r' )


        

values =list(map(lambda x:tuple(x.strip().split(' ')), f.readlines()))
h=0
d=0

for c,v in values:
    if c=='forward':
        h+=int(v)
    if c=='down':
        d+=int(v)
    if c=='up':
        d-=int(v)
print(h*d)
a=0
h=0
d=0
for c,v in values:
    if c=='forward':
        h+=int(v)
        d+=a*int(v)
    if c=='down':
        a+=int(v)
    if c=='up':
        a-=int(v)
print(h*d)

1840243
1727785422


# 3

In [63]:
from helper import aoc
import numpy as np
year=2021
day=3
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )
values=np.array(list(map(lambda x:list(map(int,list(x.strip()))), f.readlines())))
# with np.printoptions(threshold=np.inf):
#     print(values[:,0])
#     print(values.T[0,:])

maxOcc=''
minOcc=''
for v in values.T:
    maxOcc+=str(np.bincount(v).argmax())
    minOcc+=str(np.bincount(v).argmin())
print(int(maxOcc,2)*int(minOcc,2))

def recursiveFilterByOcc(data,i,tiebreaker,func):
    maxOcc=[]
    occurences=np.bincount(data[i,:])
    
    # print(occurences)
    # if(occurences[0]==1):
    # print('data:{}'.format(data[i,:]))
    if(not sum(occurences)==1):
        if(len(occurences)==2 and occurences[0]==occurences[1]):
            choice=tiebreaker
        else:
            choice=func(occurences)
    else:
        choice=data[i,0]

    #print(choice)
    maxOcc.append(choice)
    if(data.shape[0]<=(i+1)): return maxOcc
    fltr=np.asarray(maxOcc)
    filteredData=data.T[np.in1d(data.T[:,i],fltr)].T
    return maxOcc + recursiveFilterByOcc(filteredData,i+1,tiebreaker,func)

#print(maxOcc)
print(int(''.join(map(str,recursiveFilterByOcc(values.T,0,1,np.argmax))),2)*int(''.join(map(str,recursiveFilterByOcc(values.T,0,0,np.argmin))),2))


1025636
793873


# 4

In [61]:
from helper import aoc
import numpy as np
import re

year=2021
day=4
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

numbersChosen=list(map(int,f.readline().strip().split(',')))
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

class BingoBoard:

    def __init__(self,grid) -> None:
        self.grid=grid
        self.mask=np.full(grid.shape,False)
        self.lastNumberTickedOff=0
    def reset(self):
        self.mask=np.full(grid.shape,False)
        self.lastNumberTickedOff=0
    def isBingo(self):
        
        for row in range(self.mask.shape[0]):
            if(all(self.mask[row,:]==True)):
                return True
        for col in range(self.mask.shape[1]):
            if(all(self.mask[:,col]==True)):
                return True
        
        # if(all(self.mask.diagonal()==True)):
        #     return True
        # if(all(np.fliplr(self.mask).diagonal()==True)):
        #     return True
        return False
    
    def getUntickedSum(self):
        result=np.invert(self.mask)*self.grid
        #print(result)
        return result.sum()*self.lastNumberTickedOff
    
    def tickOff(self,number:int):
        found=np.argwhere(self.grid==number)
        if not found.size==0:
            self.mask[found[0][0],found[0][1]]=True
            self.lastNumberTickedOff=number
        bingo=self.isBingo()
        # if bingo:
        #     print(self.getUntickedSum())
        return bingo
    def __repr__(self) -> str:
        r=''
        row=0
        for i,x in np.ndenumerate(self.grid):
            if i[0]>row:
                r+='\n'
                row=i[0]
            # print(i)    
            if self.mask[i]:
                if x==self.lastNumberTickedOff:
                    r+=color.BLUE
                else:
                    r+=color.RED
            r+='{:3d}'.format(x)
            if self.mask[i]:
                r+=color.END
        
        return r



bingoRows=[]
bingoSets=[]
for l in f.readlines():
    l=l.strip()
    if l=='':
        if(len(bingoRows)!=0):

            bingoSets.append(BingoBoard(np.array(bingoRows)))
            bingoRows=[]
        continue
    else:
        numbers=list(map(int,re.findall('\d{1,2}',l)))
        bingoRows.append(numbers)

# for nc in numbersChosen:
#     if any(map(lambda x: x.tickOff(nc),bingoSets)):
#         break


# print(bingoSets[0])
# for n in [99,97,50,65,79,51,56]:
#     print(bingoSets[0].tickOff(n))
#     #print(bingoSets[0])

#bingoSets[0].getUntickedSum()
for b in bingoSets:
    b.reset
isFirst=True
for nc in numbersChosen:
    bingoSetsToPlay=[x for x in bingoSets if not x.isBingo()]
    for b in bingoSetsToPlay:
        if b.tickOff(nc) and isFirst:
            print(b.getUntickedSum())
            # print(b)
            isFirst=False
    if len(bingoSetsToPlay)==1 and bingoSetsToPlay[0].isBingo():
        print(bingoSetsToPlay[0].getUntickedSum())
        # print(bingoSetsToPlay[0])
        break


16674
7075


# 5

In [60]:
from helper import aoc
import numpy as np
import re
year=2021
day=5
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

def getCoords(x):
    x=x.strip()
    match=re.match('(\d+),(\d+) \-> (\d+),(\d+)',x)
    coords=list(map(int,match.groups()))
    return coords

coords=list(map(getCoords,f.readlines()))
coords=np.array(coords)
grid=np.full((coords[:,[1,3]].max()+1,coords[:,[0,2]].max()+1),0)

hvcoords=[c for c in coords if c[0]==c[2] or c[1]==c[3]]
dcoords=[c for c in coords if not c[0]==c[2] and not c[1]==c[3]]

for c in hvcoords:
    #print(c)
    if c[0]==c[2]:
        r=[c[1],c[3]]
        r.sort()
        grid[r[0]:r[1]+1,c[0]]+=1
    elif c[1]==c[3]:
        col=[c[0],c[2]]
        col.sort()
        grid[c[1],col[0]:col[1]+1]+=1

#grid[grid==0]='.'
# for l in grid:
#     print(''.join(map(str,l)))
print(len(grid[grid>1]))

for c in dcoords:
    # print(c)
    for col,row in zip(range(c[0],c[2]-1 if c[0]>c[2] else c[2]+1,-1 if c[0]>c[2] else 1),range(c[1],c[3]-1 if c[1]>c[3] else c[3]+1,-1 if c[1]>c[3] else 1)):
        grid[row,col]+=1

# for l in grid:
#     print(''.join(map(str,l)))
print(len(grid[grid>1]))

4421
18674


# 6

In [1]:
from helper import aoc
year=2021
day=6
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 7

In [1]:
from helper import aoc
year=2021
day=7
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 8

In [1]:
from helper import aoc
year=2021
day=8
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 9

In [1]:
from helper import aoc
year=2021
day=9
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 10

In [1]:
from helper import aoc
year=2021
day=10
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 11

In [1]:
from helper import aoc
year=2021
day=11
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 12

In [1]:
from helper import aoc
year=2021
day=12
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 13

In [1]:
from helper import aoc
year=2021
day=13
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 14

In [1]:
from helper import aoc
year=2021
day=14
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 15

In [1]:
from helper import aoc
year=2021
day=15
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 16

In [1]:
from helper import aoc
year=2021
day=16
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 17

In [1]:
from helper import aoc
year=2021
day=17
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 18

In [1]:
from helper import aoc
year=2021
day=18
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 19

In [1]:
from helper import aoc
year=2021
day=19
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 20

In [1]:
from helper import aoc
year=2021
day=20
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 21

In [1]:
from helper import aoc
year=2021
day=21
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 22

In [1]:
from helper import aoc
year=2021
day=22
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 23

In [1]:
from helper import aoc
year=2021
day=23
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 24

In [1]:
from helper import aoc
year=2021
day=24
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )